In [0]:
#!eval-notebook "../Initialization/InitSystemorphToMemory"

In [0]:
Workspace.Reset(x => x.ResetInitializationRules().ResetCurrentPartitions());
Workspace.Initialize(x => x.FromSource(DataSource));
var defaulttRawVarsBE = await DataSource.Query<RawVariable>().ToArrayAsync();
var defaultVarsBE = await Workspace.Query<IfrsVariable>().ToArrayAsync();

In [0]:
var yieldCurve_Test = @"
@@YieldCurve
Currency,Year,Month,Scenario,Values0,Values1,Values2,Values3
USD,2021,6,,0.003,0.003,0.003,0.003
USD,2021,6,MTUP10pct,0.103,0.103,0.103,0.103";

In [0]:
var cashflows_BestEstimate = @"
@@Main
ReportingNode,Year,Month,Scenario
CH,2021,6,
@@Cashflow
DataNode,AmountType,AocType,Novelty,AccidentYear,Values0,Values1,Values2,Values3,Values4,Values5,Values6,Values7,Values8,Values9,Values10,Values11,Values12,Values13,Values14,Values15,Values16,Values17,Values18,Values19,Values20,Values21,Values22,Values23
DT1.1,PR,CL,C,,110,0,0,110,0,0,110,0,0,110,0,0,0,110,0,0,110,0,0,110,0,0,110,0
DT1.1,NIC,CL,C,,-27.5,-27.5,-27.5,-27.5,-27.5,-27.5,-27.5,-27.5,-27.5,-27.5,-27.5,-27.5,-27.5,-27.5,-27.5,-27.5,-27.5,-27.5,-27.5,-27.5,-27.5,-27.5,-27.5,-27.5
DT1.1,CU,CL,C,,-11,-11,-11,-11,-11,-11,-11,-11,-11,-11,-11,-11,-11,-5.5,-5.5,-5.5,-5.5,-5.5,-5.5,-5.5,-5.5,-5.5,-5.5,-3.3
DT1.1,RA,CL,C,,-2.75,-2.75,-2.75,-2.75,-2.75,-2.75,-2.75,-2.75,-2.75,-2.75,-2.75,-2.75,-2.75,-2.75,-2.75,-2.75,-2.75,-2.75,-2.75,-2.75,-2.75,-2.75,-2.75,-2.75";

In [0]:
var actuals_BestEstimate = @"
@@Main
ReportingNode,Year,Month,Scenario
CH,2021,6,
@@Actual
DataNode,AocType,ValueType,AccidentYear,Value
DT1.1,CF,NIC,,-3008";

In [0]:
var argsBestEstimate = new ImportArgs ("CH", 2021, 6, Periodicity.Quarterly, null, ImportFormats.Cashflow);
var argsScenario = new ImportArgs ("CH", 2021, 6, Periodicity.Quarterly, "MTUP10pct", ImportFormats.Cashflow);

In [0]:
var yieldCurveLog = await Import.FromString(yieldCurve_Test).WithFormat(ImportFormats.YieldCurve).WithTarget(DataSource).ExecuteAsync();
Workspace.Reset(x => x.ResetInitializationRules().ResetCurrentPartitions());
Workspace.Initialize(x => x.FromSource(DataSource));
yieldCurveLog

# Import Cashflow

In [0]:
await Import.FromString(cashflows_BestEstimate).WithFormat(ImportFormats.Cashflow).WithTarget(DataSource).ExecuteAsync()

In [0]:
await DataSource.Partition.SetAsync<PartitionByReportingNode>(argsBestEstimate);
await DataSource.Partition.SetAsync<PartitionByReportingNodeAndPeriod>(argsBestEstimate);
var rawVarsBE = await DataSource.Query<RawVariable>().ToArrayAsync();
var ifrsVarsBE = await DataSource.Query<IfrsVariable>().ToArrayAsync();
rawVarsBE.Length.Should().NotBe(0);
ifrsVarsBE.Length.Should().NotBe(0);

In [0]:
await DataSource.Partition.SetAsync<PartitionByReportingNode>(argsScenario);
await DataSource.Partition.SetAsync<PartitionByReportingNodeAndPeriod>(argsScenario);
var rawVarsSCE = await DataSource.Query<RawVariable>().ToArrayAsync();
var ifrsVarsSCE = await DataSource.Query<IfrsVariable>().ToArrayAsync();
rawVarsSCE.Length.Should().Be(0);
ifrsVarsSCE.Length.Should().NotBe(0);

In [0]:
ifrsVarsSCE.Where(x => x.EconomicBasis == "C").Intersect(ifrsVarsBE.Where(x => x.EconomicBasis == "C"), IfrsVariableComparer.Instance()).ToArray().Length.Should().Be(0);

# Import Actuals

In [0]:
await Import.FromString(actuals_BestEstimate).WithFormat(ImportFormats.Actual).WithTarget(DataSource).ExecuteAsync()

In [0]:
await DataSource.Partition.SetAsync<PartitionByReportingNode>(argsBestEstimate);
await DataSource.Partition.SetAsync<PartitionByReportingNodeAndPeriod>(argsBestEstimate);
var newRawVarsBE = await DataSource.Query<RawVariable>().ToArrayAsync();
var newIfrsVarsBE = await DataSource.Query<IfrsVariable>().ToArrayAsync();
newRawVarsBE.Length.Should().NotBe(0);
newIfrsVarsBE.Length.Should().NotBe(0);

In [0]:
newIfrsVarsBE.Except(ifrsVarsBE, IfrsVariableComparer.Instance()).ToArray().Length.Should().NotBe(0);